This Jupyter Notebook's goal is to clean up one large customer datasets. This dataset will eventually be used to predict whether some students are looking for a new job or not, information that some other team will then use to direct them to prospective recruiters.

"student_subset.csv", is a subset of their entire student dataset, 
This is a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `student_id`   | A unique ID for each student.                 |
| `city`  | A code for the city the student lives in.  |
| `city_development_index` | A scaled development index for the city.       |
| `gender` | The student's gender.       |
| `relevant_experience` | An indicator of the student's relevant experience.       |
| `enrolled_university` | The type of university course enrolled in (if any).       |
| `education_level` | The student's education level.       |
| `major_discipline` | The educational discipline of the student.       |
| `experience` | The student's total experience (in years).       |
| `company_size` | The number of employees at the student's current employer.       |
| `last_new_job` | The number of years between the student's current and previous jobs.       |
| `training_hours` | The number of hours of training completed.       |
| `job_change` | An indicator of whether the student is looking for a new job (`1`) or not (`0`).       |

In [1]:
# Start your code here!
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('student_subset.csv', dtype={'city':'category', 'gender':'category','relevant_experience':'category','enrolled_university':'category','major_discipline':'category','company_type':'category'})

In [3]:
df.head(10)

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0
5,21651,city_176,0.764,NaN,Has relevant experience,Part time course,Graduate,STEM,11,NaN,NaN,1,24,1.0
6,28806,city_160,0.920,Male,Has relevant experience,no_enrollment,High School,NaN,5,50-99,Funded Startup,1,24,0.0
7,402,city_46,0.762,Male,Has relevant experience,no_enrollment,Graduate,STEM,13,<10,Pvt Ltd,>4,18,1.0
8,27107,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,7,50-99,Pvt Ltd,1,46,1.0
9,699,city_103,0.920,NaN,Has relevant experience,no_enrollment,Graduate,STEM,17,10000+,Pvt Ltd,>4,123,0.0


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              19158 non-null  int32   
 1   city                    19158 non-null  category
 2   city_development_index  19158 non-null  float16 
 3   gender                  14650 non-null  category
 4   relevant_experience     19158 non-null  category
 5   enrolled_university     18772 non-null  category
 6   education_level         18698 non-null  category
 7   major_discipline        16345 non-null  category
 8   experience              19093 non-null  category
 9   company_size            13220 non-null  category
 10  company_type            13018 non-null  category
 11  last_new_job            18735 non-null  category
 12  training_hours          19158 non-null  int32   
 13  job_change              19158 non-null  float16 
dtypes: category(10), float

In [6]:
# Convert integer columns into int32
df[["student_id", "training_hours"]] = df[["student_id", "training_hours"]].astype("int32")

In [7]:
# Convert float column into float16
df[["city_development_index", "job_change"]] = df[["city_development_index", "job_change"]].astype("float16")

In [8]:
df.experience.unique()
#df.company_size.unique()

array(['>20', '15', '5', '<1', '11', '13', '7', '17', '2', '16', '1', '4',
       '10', '14', '18', '19', '12', '3', '6', '9', '8', '20', nan],
      dtype=object)

In [9]:
education = pd.CategoricalDtype(['Primary School', 'High School', 'Graduate', 'Masters', 'Phd'], ordered=True)
# Convert experience into an ordered category
exp_strings_lst = ['<1'] + list(map(str, range(1, 21))) + ['>20']
experience = pd.CategoricalDtype(exp_strings_lst, ordered=True)
company = pd.CategoricalDtype(['<10', '10-49', '50-99', '100-499', '500-999', '1000-4999', '5000-9999', '10000+'], ordered=True)
job = pd.CategoricalDtype(['never', '1', '2', '3', '4', '>4'], ordered=True)


In [10]:
df['education_level'] = df['education_level'].astype(education)
df['experience'] = df['experience'].astype(experience)
df['company_size'] = df['company_size'].astype(company)
df['last_new_job'] = df['last_new_job'].astype(job)


In [13]:
ds_clean = df[(df['experience']>='10') & (df['company_size'] >= '1000-4999')]

In [14]:
ds_clean.head(100)

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
9,699,city_103,0.919922,NaN,Has relevant experience,no_enrollment,Graduate,STEM,17,10000+,Pvt Ltd,>4,123,0.0
12,25619,city_61,0.913086,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,1000-4999,Pvt Ltd,3,23,0.0
31,22293,city_103,0.919922,Male,Has relevant experience,Part time course,Graduate,STEM,19,5000-9999,Pvt Ltd,>4,141,0.0
34,26494,city_16,0.910156,Male,Has relevant experience,no_enrollment,Graduate,Business Degree,12,5000-9999,Pvt Ltd,3,145,0.0
40,2547,city_114,0.925781,Female,Has relevant experience,Full time course,Masters,STEM,16,1000-4999,Public Sector,2,14,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
920,28453,city_16,0.910156,Male,Has relevant experience,no_enrollment,Masters,STEM,10,10000+,Pvt Ltd,1,18,0.0
931,6304,city_103,0.919922,Male,Has relevant experience,Part time course,Graduate,STEM,19,1000-4999,Public Sector,>4,142,0.0
933,31368,city_103,0.919922,Male,Has relevant experience,no_enrollment,Graduate,STEM,10,1000-4999,Pvt Ltd,1,314,0.0
959,21276,city_75,0.938965,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,10000+,Pvt Ltd,4,94,0.0
